# Yahoo Fantasy Football Data Pulls

### Set up

In [ ]:
import pandas as pd
import numpy as np
import logging
import os
import pprint
import warnings
import json
import csv
from datetime import datetime

import pytest
from dotenv import load_dotenv

from yfpy import Data
from yfpy.models import Game, StatCategories, User, Scoreboard, Settings, Standings, League, Player, Team, \
    TeamPoints, TeamStandings, Roster
from yfpy.query import YahooFantasySportsQuery

In [ ]:
# Settings from yfpy package
logging.getLogger("yfpy.query").setLevel(level=logging.INFO)
warnings.simplefilter("ignore", ResourceWarning)
env_path = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath('private.json'))), ".env")
load_dotenv(dotenv_path=env_path)
print_output = True 
browser_callback = True
auth_dir = "."
data_dir = os.path.join(os.path.dirname(os.path.abspath('private.json')), "test_output")

season = '2019'
game_code = "nfl"
if season == '2019':
    game_key = "390"
    league_id = "1021512"
elif season == '2020':
    game_key = "399"
    league_id = "234883"

with open(r'C:/Users/bradr/Python/fantasy_football/token.json', 'r') as json_file:
    private = json.load(json_file)

# Instantiate yfpy objects
yahoo_data = Data(data_dir)
yahoo_query = YahooFantasySportsQuery(
    auth_dir,
    league_id,
    game_id=game_key,
    game_code=game_code,
    offline=False,
    all_output_as_json=False,
    consumer_key=private['consumer_key'],
    consumer_secret=private['consumer_secret'],
    browser_callback=browser_callback
)

Keys for other seasons

In [ ]:
# For other season lookups
# yahoo_query.get_all_yahoo_fantasy_game_keys()

# Draft

### Picks

In [ ]:
draft_data = yahoo_query.get_league_draft_results()
draft_picks = []
for pick in range(0, len(draft_data)):
    draft_picks.append((draft_data[pick]['draft_result'].pick,
                        draft_data[pick]['draft_result'].round,
                        draft_data[pick]['draft_result'].team_key[-1:],
                        draft_data[pick]['draft_result'].player_key.split(".")[2]))
draft_picks_df = pd.DataFrame(draft_picks, columns = ['pick','round','team_key','player_key'])
draft_picks_df.to_csv(f'output/{season}_draft_picks.csv', index = False)
# draft_picks_df

### Player Info

In [ ]:
draft_player_list = draft_picks_df['player_key'].tolist()
player_info = []
for player in draft_player_list:
    player_draft = yahoo_query.get_player_draft_analysis(player_key = f'{game_key}.p.' + player)
    player_info.append((player_draft.player_key.split(".")[2],
                        player_draft.name.full,
                        player_draft.display_position,
                        player_draft.draft_analysis.average_pick,
                        player_draft.draft_analysis.average_round,
                        player_draft.draft_analysis.average_cost,
                        player_draft.draft_analysis.percent_drafted,
                        player_draft.bye_weeks.week,
                        player_draft.headshot.url))
player_info_df = pd.DataFrame(player_info, columns = ['player_key',
                                                      'name',
                                                      'position',
                                                      'average_pick',
                                                      'average_round',
                                                      'averago_cost',
                                                      'percent_drafted',
                                                      'bye_weaaek',
                                                      'image'])
player_info_df.to_csv(f'output/{season}_player_draft_info.csv', index = False)
# player_info_df

# Team

In [ ]:
standings_data = yahoo_query.get_league_standings()
	
# Convert Standings Object to json
parsed_json = (json.loads(str(standings_data)))

# Initialize a list that can store the data
team_list = []
# Iterate over the Standings json to get teamids and their respective points
for team in parsed_json['teams']:
    team_list.append((int(team['team']['team_id']),
                      team['team']['name'],
                      team['team']['team_key'],
                      team['team']['draft_grade'],
                      team['team']['team_logos']['team_logo']['url']))

# Sort the team list by the team id
team_list = sorted(team_list)
teams = pd.DataFrame(team_list, columns = ['team_id','team_name','team_key','draft_grade','team_logo'])
teams.to_csv(f'output/{season}_teams.csv', index = False)
# teams

# Rosters

### Team rosters by Week
This gets the rosters for each team for each week of the season and puts them into a dataframe

Potentially add the stats as columns?

In [ ]:
roster_list = []
for week in range(1,17):
    for league_team in range(1,11): 
        roster_week = yahoo_query.get_team_roster_player_info_by_week(team_id = league_team, chosen_week = week)
        for player in range(0,len(roster_week)):
            roster_list.append((league_team,
                                week,
                                roster_week[player]['player'].display_position, 
                                roster_week[player]['player'].selected_position.position,
                                roster_week[player]['player'].full_name,
                                roster_week[player]['player'].player_key,
                                roster_week[player]['player'].player_points_value,
                                roster_week[player]['player'].headshot_url))
    
roster_week_df = pd.DataFrame(roster_list, columns = ['team_id',
                                                      'week',
                                                      'position',
                                                      'lineup_position',
                                                      'full_name',
                                                      'player_key',
                                                      'fantasy_points',
                                                      'player_pic'])
roster_week_df.to_csv(f'output/{season}_roster_week.csv', index = False)
# roster_week_df

# Points

### Player Points By Week

Done with nflscrapR in R so that I can get it for free agents and non draft picks

### Weekly Matchup Scores

In [ ]:
weekly = []
for week in range(1,17):
    scoreboard = yahoo_query.get_league_scoreboard_by_week(chosen_week = week)
    for game in range(0,len(scoreboard.matchups)):
        weekly.append((week,
                       scoreboard.matchups[game]['matchup'].teams[0]['team'].team_id,
                       scoreboard.matchups[game]['matchup'].teams[0]['team'].name.decode('utf-8'),
                       scoreboard.matchups[game]['matchup'].teams[0]['team'].team_projected_points.total,
                       scoreboard.matchups[game]['matchup'].teams[0]['team'].team_points.total,
                       scoreboard.matchups[game]['matchup'].matchup_grades[0]['matchup_grade'].grade,
                       scoreboard.matchups[game]['matchup'].teams[1]['team'].team_id,
                       scoreboard.matchups[game]['matchup'].teams[1]['team'].name.decode('utf-8'),
                       scoreboard.matchups[game]['matchup'].teams[1]['team'].team_projected_points.total,
                       scoreboard.matchups[game]['matchup'].teams[1]['team'].team_points.total,
                       scoreboard.matchups[game]['matchup'].matchup_grades[1]['matchup_grade'].grade,
                       scoreboard.matchups[game]['matchup'].week_start,
                       scoreboard.matchups[game]['matchup'].week_end,
                       scoreboard.matchups[game]['matchup'].winner_team_key[-1:],
                       scoreboard.matchups[game]['matchup'].is_playoffs,
                       scoreboard.matchups[game]['matchup'].is_consolation,
                       scoreboard.matchups[game]['matchup'].is_tied))
    
scoreboard_df = pd.DataFrame(weekly, columns = ['week',
                                                'team_id_1',
                                                'team_name_1',
                                                'projected_points_1',
                                                'total_points_1',
                                                'matchup_grade_1',
                                                'team_id_2',
                                                'team_name_2',
                                                'projected_points_2',
                                                'total_points_2',
                                                'matchup_grade_2',
                                                'week_start',
                                                'week_end',
                                                'winner_key',
                                                'is_playoffs',
                                                'is_consolation',
                                                'is_tied'])

scoreboard_df.to_csv(f'output/{season}_weekly_scoreboard.csv', index = False)
#scoreboard_df

# Transactions

### By Team

In [ ]:
transactions_list = []
transactions = yahoo_query.get_league_transactions()

num_player_list = []
for move in range(0,len(transactions)):
    num_player_list.append(len(transactions[move]['transaction'].players))
max_num_player = max(num_player_list)

transaction_comp_cols = ['transaction_id',
                         'timestamp',
                         'type',
                         'status']

na_values = ('na','na','na','na')

for move in range(0,len(transactions)):
    transaction_components = (transactions[move]['transaction'].transaction_id,
                          datetime.utcfromtimestamp(int(transactions[move]['transaction'].timestamp)).strftime('%Y-%m-%d %H:%M:%S'),
                          transactions[move]['transaction'].type,
                          transactions[move]['transaction'].status)
    if len(transactions[move]['transaction'].players) == 1:
        transactions_list.append(transaction_components+(
                                  transactions[move]['transaction'].players['player'].transaction_data.source_type,
                                  transactions[move]['transaction'].players['player'].transaction_data.destination_team_key,
                                  transactions[move]['transaction'].players['player'].transaction_data.destination_team_name,
                                  transactions[move]['transaction'].players['player'].player_id,
                                  transactions[move]['transaction'].players['player'].name.full
                                  )) 

    for num in range(2,max_num_player+1):
        
        if len(transactions[move]['transaction'].players) == num:
            player_cols = ()
            player_col_names = []
            for i in range(0,len(transactions[move]['transaction'].players)):
                player_values = (transactions[move]['transaction'].players[i]['player'].transaction_data.source_type,
                                 transactions[move]['transaction'].players[i]['player'].transaction_data.destination_team_key,
                                 transactions[move]['transaction'].players[i]['player'].transaction_data.destination_team_name,
                                 transactions[move]['transaction'].players[i]['player'].player_id,
                                 transactions[move]['transaction'].players[i]['player'].name.full)
                player_cols = player_cols + player_values
                j = i + 1
                player_col_names = player_col_names + [f'source_type_{j}',
                                                       f'destination_team_key_{j}',
                                                       f'destination_team_name_{j}',
                                                       f'player_id_{j}',
                                                       f'player_name_{j}']
                
                if num == max_num_player:
                    final_player_col_names = player_col_names

            transactions_list.append(transaction_components+player_cols)  
        

transactions_df = pd.DataFrame(transactions_list, columns = transaction_comp_cols+list(final_player_col_names))
transactions_df.to_csv(f'output/{season}_transactions.csv', index = False)
# transactions_df

### By Player